In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os

In [2]:
# set folder path

path = r'C:\Users\shyro\OneDrive\Documents\Instacart Basket Analysis'

In [3]:
# Import ords_prods_merged.pkl

ords_prods_merge = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merged.pkl'))

In [5]:
#  aggregate mean of the “order_number” column grouped by “department_id” for the entire dataframe

ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


#### Observation: the entire dataframe is able to display the mean for 21 department_id as opposed to 8 department_id from the subset. In addition, the mean for each department is smaller in the entire dataframe rather than the subset because there were more values being calculated. 

## Creating a loyalty flag for existing customers using the transform() and loc() functions.

### Step 1: Split the data into groups based on the “user_id” column.

### Step 2: Apply the transform() function on the “order_number” column to generate the maximum orders for each user.

In [6]:
ords_prods_merge['max_order'] = ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [7]:
# check output 

ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours_of_day,max_order
0,2539329,1,prior,1,2,8,NaN,196,1,0,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
1,2398795,1,prior,2,3,7,15.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10
2,473747,1,prior,3,3,12,21.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10
3,2254736,1,prior,4,4,7,29.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10
4,431534,1,prior,5,4,15,28.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10
5,3367565,1,prior,6,2,7,19.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10
6,550135,1,prior,7,1,9,20.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10
7,3108588,1,prior,8,1,14,14.0,196,2,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10
8,2295261,1,prior,9,1,16,0.0,196,4,1,Soda,77,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10
9,2550362,1,prior,10,4,8,30.0,196,1,1,Soda,77,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10


### Step 3: Create a loyalty flag using loc

In [8]:
ords_prods_merge.loc[ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [9]:
ords_prods_merge.loc[(ords_prods_merge['max_order'] <= 40) & (ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [10]:
ords_prods_merge.loc[ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [12]:
# check the count of loyalty_flags

ords_prods_merge['loyalty_flag'].value_counts(dropna = False)

Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: loyalty_flag, dtype: int64

In [13]:
# check the shape of ords_prods_merge

ords_prods_merge.shape

(32404859, 21)

##  Use the loyalty flag you created and check the basic statistics of the product prices for each loyalty category (Loyal Customer, Regular Customer, and New Customer)

In [16]:
# performing multiple aggregation

ords_prods_merge.groupby('loyalty_flag').agg({'prices': ['mean', 'min', 'max', 'sum']})

prices                           
                       mean  min      max          sum
loyalty_flag                                          
Loyal customer    10.386336  1.0  99999.0  106814042.2
New customer      13.294670  1.0  99999.0   83011787.2
Regular customer  12.495717  1.0  99999.0  198391693.2

### Create a spending flag for each user based on the average price across all their orders using the following criteria:
 - If the mean of the prices of products purchased by a user is lower than 10, then flag them as a “Low spender.”
 - If the mean of the prices of products purchased by a user is higher than or equal to 10, then flag them as a “High spender.”


### Step 1: Split the data into groups based on the “user_id” column.

### Step 2: Apply the transform() function on the “prices” column to generate the average prices for each user.

In [17]:
ords_prods_merge['avg_prices'] = ords_prods_merge.groupby(['user_id'])['prices'].transform(np.max)

In [18]:
# check output 

ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,department_id,prices,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours_of_day,max_order,loyalty_flag,avg_prices
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,14.0
2,473747,1,prior,3,3,12,21.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,14.0
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,14.0
4,431534,1,prior,5,4,15,28.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,14.0
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0
6,550135,1,prior,7,1,9,20.0,196,1,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,7,9.0,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,7,9.0,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,14.0


### Step 3: Create a spending flag using loc

In [19]:
ords_prods_merge.loc[ords_prods_merge['avg_prices'] < 10, 'spending_flag'] = 'Low spender'

In [20]:
ords_prods_merge.loc[ords_prods_merge['avg_prices'] > 10, 'spending_flag'] = 'High spender'

In [21]:
# check the count of spending_flags

ords_prods_merge['spending_flag'].value_counts(dropna = False)

High spender    32371364
Low spender        31890
NaN                 1605
Name: spending_flag, dtype: int64

### Create an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column. The criteria for the flag should be as follows:
 - If the median of “days_since_prior_order” is higher than 20, then the customer should be labeled a “Non-frequent customer.”
 - If the median is higher than 10 and lower than or equal to 20, then the customer should be labeled a “Regular customer.”
 - If the median is lower than or equal to 10, then the customer should be labeled a “Frequent customer.”

### Step 1: Split the data into groups based on the “user_id” column.

### Step 2: Apply the transform() function on the “days_since_prior_order” column to generate the median for each user.

In [22]:
ords_prods_merge['median_freq'] = ords_prods_merge.groupby(['user_id'])['days_since_prior_order'].transform(np.max)

In [23]:
# check output 

ords_prods_merge.head(15)

,order_id,user_id,eval_set,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,_merge,price_range_loc,busiest_day,busiest_days,busiest_hours_of_day,max_order,loyalty_flag,avg_prices,spending_flag,median_freq
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,30.0
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Average orders,10,New customer,14.0,High spender,30.0
2,473747,1,prior,3,3,12,21.0,196,1,1,...,both,Mid-range product,Regularly busy,Least busy,Most orders,10,New customer,14.0,High spender,30.0
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,14.0,High spender,30.0
4,431534,1,prior,5,4,15,28.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Most orders,10,New customer,14.0,High spender,30.0
5,3367565,1,prior,6,2,7,19.0,196,1,1,...,both,Mid-range product,Regularly busy,Regularly busy,Average orders,10,New customer,14.0,High spender,30.0
6,550135,1,prior,7,1,9,20.0,196,1,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0,High spender,30.0
7,3108588,1,prior,8,1,14,14.0,196,2,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0,High spender,30.0
8,2295261,1,prior,9,1,16,0.0,196,4,1,...,both,Mid-range product,Regularly busy,Busiest day,Most orders,10,New customer,14.0,High spender,30.0
9,2550362,1,prior,10,4,8,30.0,196,1,1,...,both,Mid-range product,Least busy,Least busy,Average orders,10,New customer,14.0,High spender,30.0


### Step 3: Create a spending flag using loc

In [24]:
ords_prods_merge.loc[ords_prods_merge['median_freq'] > 20, 'frequency_flag'] = 'Non-frequent customer'

In [25]:
ords_prods_merge.loc[(ords_prods_merge['median_freq'] <= 20) & (ords_prods_merge['median_freq'] > 10), 'frequency_flag'] = 'Regular customer'

In [26]:
ords_prods_merge.loc[ords_prods_merge['median_freq'] <= 10, 'frequency_flag'] = 'Frequent customer'

In [27]:
# check the count of frequency_flag

ords_prods_merge['frequency_flag'].value_counts(dropna = False)

Non-frequent customer    24487186
Regular customer          6533511
Frequent customer         1384157
NaN                             5
Name: frequency_flag, dtype: int64

In [30]:
ords_prods_merge.shape

(32404859, 25)

In [33]:
ords_prods_merge[['user_id', 'avg_prices', 'loyalty_flag', 'order_number','frequency_flag','spending_flag']].head(20)

,user_id,avg_prices,loyalty_flag,order_number,frequency_flag,spending_flag
0,1,14.0,New customer,1,Non-frequent customer,High spender
1,1,14.0,New customer,2,Non-frequent customer,High spender
2,1,14.0,New customer,3,Non-frequent customer,High spender
3,1,14.0,New customer,4,Non-frequent customer,High spender
4,1,14.0,New customer,5,Non-frequent customer,High spender
5,1,14.0,New customer,6,Non-frequent customer,High spender
6,1,14.0,New customer,7,Non-frequent customer,High spender
7,1,14.0,New customer,8,Non-frequent customer,High spender
8,1,14.0,New customer,9,Non-frequent customer,High spender
9,1,14.0,New customer,10,Non-frequent customer,High spender


### Export your dataframe as a pickle file and store it correctly in your “Prepared Data” folder.

In [35]:
ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data','ords_prods_merged_grouped_aggregated.pkl'))